In [8]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
train_tape = "Z:/rppg/scamps_datatape_32x36x36_train.h5"
valid_tape = "Z:/rppg/scamps_datatape_32x36x36_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

def to_tf_deep_phys(datatape, dtype=tf.float16):
    def _():
        for i, j in datatape:
            j = tf.concat([j[1:]-j[:-1], [0]], axis=0)
            #yield i, (j-tf.reduce_mean(j, axis=0))/(tf.math.reduce_std(j, axis=0)+1e-6)
            yield i, j
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))

#train, valid = to_tf_deep_phys(train).cache(f'{tmp}/deepphys'), to_tf_deep_phys(valid).cache(f'{tmp}/deepphys')
train, valid = to_tf_deep_phys(train), to_tf_deep_phys(valid)

In [24]:
deepphys = DeepPhys_end_to_end()
deepphys.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001), loss='mse')
deepphys.build(input_shape=(None, 36, 36, 3))
print(f'Flops per frame:{get_flops(deepphys, input_sig=[tf.TensorSpec([32, 36, 36, 3])])/32:.0f}')
deepphys.summary()

Flops per frame:53434160
Model: "deep_phys_end_to_end_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_4 (Functional)         (None, 1)                 532899    
Total params: 532,899
Trainable params: 532,899
Non-trainable params: 0
_________________________________________________________________


In [25]:
deepphys.fit(train.shuffle(2000), validation_data=valid, epochs=1, callbacks=[keras.callbacks.ModelCheckpoint('../weights/DeepPhys_SCAMPS.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
deepphys.load_weights('../weights/DeepPhys_SCAMPS.h5')

37909/37909 [==============================] - 652s 17ms/step - loss: 0.1860 - val_loss: 0.1489

Epoch 00001: val_loss improved from inf to 0.14893, saving model to ../weights\DeepPhys_SCAMPS.h5


In [14]:
deepphys.load_weights('../weights/DeepPhys_SCAMPS.h5')

In [16]:
def cumsum(result, overwrite=True):
    with h5py.File(result, 'a') as f:
        for i, j in f.items():
            if 'diff' not in j.keys():
                j.create_dataset('diff', data=j['predict'])
                j['predict'][...] = detrend(np.cumsum(j['diff']))
            elif overwrite:
                j['predict'][...] = detrend(np.cumsum(j['diff']))

In [26]:
eval_on_dataset(test_set_PURE, deepphys, 32, (36, 36), step=1, batch=0, save='../results/DeepPhys_SCAMPS_PURE.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/DeepPhys_SCAMPS_PURE.h5')
get_metrics('../results/DeepPhys_SCAMPS_PURE.h5')

100%|██████████| 59/59 [00:22<00:00,  2.68it/s]


{'Sliding window': {'MAE': 22.078, 'RMSE': 29.413, 'R': 0.12272},
 'Whole video': {'MAE': 17.626, 'RMSE': 24.109, 'R': 0.34937}}

In [27]:
eval_on_dataset(test_set_UBFC_rPPG2, deepphys, 32, (36, 36), step=1, batch=0, save='../results/DeepPhys_SCAMPS_UBFC.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/DeepPhys_SCAMPS_UBFC.h5')
get_metrics('../results/DeepPhys_SCAMPS_UBFC.h5')

100%|██████████| 42/42 [00:16<00:00,  2.53it/s]


{'Sliding window': {'MAE': 16.655, 'RMSE': 24.87, 'R': 0.31714},
 'Whole video': {'MAE': 14.177, 'RMSE': 21.187, 'R': 0.39728}}